In [ ]:
!pip install faiss-cpu iprogress ipykernel ipywidgets langchain langchain-community langchain-groq langchain-classic langchain-huggingface langchain-text-splitters pypdf python-dotenv sentence-transformers

In [ ]:
import os
import io
import getpass
from google.colab import files
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

Enter your Groq API Key: ··········


In [ ]:
llm = ChatGroq(
    temperature=0,
    model_name="llama-3.1-8b-instant",  # You can also use "mixtral-8x7b-32768"
)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


print("Upload your CV PDF file:")
uploaded = files.upload()

filename = list(uploaded.keys())[0]
print(f"Processing {filename}...")

loader = PyPDFLoader(filename)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

print(f"Success! {len(splits)} chunks created and indexed.")

#Refine prompt as per your need
system_prompt = (
    "You are an expert HR and Recruitment Assistant. "
    "You are analyzing a candidate's CV/Resume. "
    "Use the following pieces of retrieved context to answer the question. "
    "If the information is not in the CV, strictly say 'I cannot find that information in the provided CV.' "
    "Keep your answers concise and professional."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Create the chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("Chatbot is ready!")

In [ ]:
print(f"Start chatting with {filename} (Type 'exit' to stop):")
print("-" * 50)

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        break

    # Run the RAG chain
    response = rag_chain.invoke({"input": user_input})

    print(f"AI: {response['answer']}")
    print("-" * 50)